In [1]:
import torch
import torch.nn as nn
import torchvision 
import torchvision.transforms as transforms

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

#Hyper Parameter
num_epochs = 30
batch_size = 100
learning_rate = 0.001

#Image preprocessing modules
transform = transforms.Compose([
    transforms.Pad(4),
    transforms.RandomHorizontalFlip(),
    transforms.ReandomCrop(32),
    transforms.ToTensor()
])

#CIFAR-10 dataset
train_dataset = torchvision.datasets.CIFAR10(
    root='../../data',
    train = True,
    transforms = transform,
    download = True
)

test_dataset = torchvision.datasets.CIFAR10(
    root='../../data',
    train = False,
    transforms = transforms.ToTensor()
)

#Data loader
train_loader = torch.utils.data.Dataloader(dataset= train_dataset,
                                           batch_size= batch_size,
                                           shuffle= True)

test_loader = torch.utils.data.Dataloader(dataset= test_dataset,
                                           batch_size= batch_size,
                                           shuffle= False)

In [ ]:
def conv3x3(in_channels, out_channels, stride =1):
    return nn.Conv2d(in_channels, out_channels, kernel_size=3, stride=stride, padding =1, bias=False)

class ResidualBlock(nn.Module):
    def __init__(self, in_channels, out_channels, stride =1, downsampling= None):
        super(ResidualBlock, self).__init__()
        self.conv1 = conv3x3(in_channels, out_channels, stride)
        self.bn1 = nn.BatchNorm2d(out_channels)
        self.relu = nn.Relu(inplace = True)
        self.conv2 = conv3x3(in_channels, out_channels, stride = 1) #stride=1 no downsampling
        self.bn2 = nn.BatchNorm2d(out_channels)
        self.downsampling = downsampling

    def forward(self, x):
        residual = x.clone
        
        out = self.conv1(x)
        out = self.bn1(out)
        out = self.relu(out)
        out = self.conv2(x)
        out = self.bn2(out)

        if self.downsampling:
            residual = self.downsampling(x)

        out += residual

        out = self.relu(out)
        
        return out


In [ ]:
class ResNet(nn.Module):
    def __init__(self, block, layers, num_classes = 10):
        """
        layers will be a list: [2,2,2] 
        # we have 3 layers -> Lenght odf the list
        # the elements within the list specify how many residual blocks we have in each layer
        """
        super(ResNet, self).__init__()
        self.input_channels = 16
        self.conv = conv3x3(3,16) #3 ->we have got an RGB image
        self.bn = nn.BatchNorm2d(16)
        self.relu = nn.Relu(inplace = True)
        self.layer1 = self.make_layer(block,  16, layers[0], stride =1)
        self.layer2 = self.make_layer(block,  32, layers[1], stride =2) #downsampling
        self.layer3 = self.make_layer(block,  64, layers[2], stride =2) #downsampling
        self.avg_pool =  nn.AvgPool(8) #kernel size is equals to the size of the feature map -> we start with 32x32 image size and we have 2 strides
        self.fc = nn.Linear(64, num_classes) 

    def make_layer(self, block, out_channels, blocks, stride =1 ):
        downsampling = None
        if (self.input_channels!= out_channels) or (stride !=1 ):
            downsampling = nn.Sequential( conv3x3(self.input_channels, out_channels, stride=stride),
                                          nn.BatchNorm2d(out_channels))  # So it's not together, but I mean after we finish from this layer, then we run this layer.

        residual_blocks = []
        residual_blocks.append(block(self.input_channels, out_channels, stride = stride, downsampling = downsampling))
        self.input_channels = out_channels
        residual_blocks.append(block(self.input_channels, out_channels))
        return nn.Sequential(*residual_blocks)

    def forward(self, x):
        out = self.conv(x)
        out = self.bn(out)
        out = self.relu(out)
        out = self.layer1(out)
        out =self.layer2(out)
        out = self.layer3(out)
        out = self.avg_pool(out)
        out = out.view(out.size(0), -1)
        out = self.fc(out)
        return out

In [ ]:
model = ResNet(ResidualBlock, [2,2,2])
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr = learning_rate)

In [ ]:
decay = 0
model.train() #use batchnoralization
for epoch in range(num_epochs):
    # Decay the learning rate by a factor of 0.5 every 20 epochs
    if (epoch +1) % 20 ==0 :
        decay +=1
        optimizer.param_groups[0]['lr'] = learning_rate * (0.5**decay)
        print("The new learning rate is {}".format(optimizer.param_groups[0]['lr']))

    for i,(images,labels) in enumerate(train_loader):
        images = images.to(device)
        labels =labels.to(device)
        outputs = model(images)
        loss = criterion(outputs,labels)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step() #update the weights

        if (i+1) % 100 == 0:
            print("Epoch [{}/{}], Step [{}/{}], Loss{:.4f} ".format(epoch+1, num_epochs,i+1, len(train_loader), loss.item()))

In [ ]:
model.eval()
with torch.no_grad(): #no gradience computation
    correct =0
    total = 0
    for images , labels in test_loader:
        images = images.to(device)
        labels = labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().items()

    print("Accuracy of the model on the test images: {}%".format(100*correct/total))
    